# Classification NBA Model

## Configuration

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from nba_ou.data_preparation.missing_data.clean_df_for_training import (
    clean_dataframe_for_training
)
from sklearn.model_selection import KFold, cross_validate, train_test_split
from xgboost import XGBClassifier


## Load Data

In [2]:
data_path = "/home/adrian_alvarez/Projects/NBA_over_under_predictor/data/train_data/"
name = "all_odds_training_data_until_20260214.csv"

path = data_path + name

df_stats = pd.read_csv(path)

dtype_dict = {col: str for col in df_stats.columns if "ID" in col.upper()}

df_stats = pd.read_csv(
    path,
    dtype=dtype_dict
)
df_stats['GAME_DATE'] = pd.to_datetime(df_stats['GAME_DATE']).dt.strftime('%Y-%m-%d')

/tmp/ipykernel_2009712/2336588694.py:6: DtypeWarning: Columns (487,530,573,616,659,702,1178,1221,1264,1307,1350,1393) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(path)
/tmp/ipykernel_2009712/2336588694.py:10: DtypeWarning: Columns (487,530,573,616,659,702,1178,1221,1264,1307,1350,1393) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(


In [3]:
# #print column names with name in lower case
# for col in df_stats.columns:
#     if not col.isupper():
#         print(col)

In [4]:
#filter for last two seasons, gett seasons 2025 and 2024
# df_stats = df_stats[df_stats['SEASON_YEAR'].isin([2024, 2025])]

In [5]:
df_stats

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY
0,236.0,1610612762,Utah,UTA,Utah Jazz,UTA vs. POR,56,1610612757,Portland,POR,...,0.880561,0,0.588999,0.103152,-0.063841,60.319722,56.753241,0,2,0
1,214.5,1610612760,Oklahoma City,OKC,Oklahoma City Thunder,OKC vs. MIL,56,1610612749,Milwaukee,MIL,...,0.000000,0,0.532833,0.482816,-0.014822,59.817134,56.279705,0,2,0
2,237.5,1610612747,Los Angeles,LAL,Los Angeles Lakers,LAL vs. DAL,54,1610612742,Dallas,DAL,...,0.922837,0,0.444237,0.279992,0.082916,62.986122,57.349334,0,2,0
3,223.0,1610612761,Toronto,TOR,Toronto Raptors,TOR vs. DET,55,1610612765,Detroit,DET,...,0.768000,1,0.119060,0.100250,-0.022589,56.034011,57.708603,0,2,0
4,220.5,1610612744,Golden State,GSW,Golden State Warriors,GSW vs. SAS,55,1610612759,San Antonio,SAS,...,0.864772,1,0.517551,0.058655,-0.107387,60.054287,57.981278,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11036,NaN,1610612762,Utah,UTA,Utah Jazz,UTA vs. DEN,1,1610612743,Denver,DEN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
11037,NaN,1610612764,Washington,WAS,Washington Wizards,WAS vs. PHI,1,1610612755,Philadelphia,PHI,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
11038,NaN,1610612754,Indiana,IND,Indiana Pacers,IND vs. BKN,1,1610612751,Brooklyn,BKN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
11039,NaN,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. BOS,1,1610612738,Boston,BOS,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0


In [ ]:

df_to_train = clean_dataframe_for_training(df_stats, nan_threshold=5, drop_all_na_rows=True, create_missing_flags=False, verbose=1)


STARTING DATAFRAME CLEANING PIPELINE
Starting basic cleaning with 11041 rows
Basic cleaning complete: 8427 rows remaining

Starting advanced column cleaning with 1733 columns

Advanced column cleaning complete: 1733 → 910 columns (823 removed)


Dropping NA rows for SEASON_YEAR 2017...
   Removed 0 rows with NaN values from 2017 season

Applying missing data policy...

Missing Data Policy Report:
  Rows dropped: 0 (0.0%)
  Critical columns requiring data: 5
  Columns zero-filled: 132
  Infer pairs applied: 16/228
  Remaining NaN cells: 35044

Dropping rows that are all NaN...
CLEANING COMPLETE
Final shape: (7199, 910)


In [23]:
# Count NAs per column
na_counts = df_to_train.isna().sum()

# Get most common SEASON_YEAR for nulls in each column
most_common_season = []
for col in df_to_train.columns:
    if na_counts[col] > 0:
        # Get rows where this column is null
        null_rows = df_stats[df_stats[col].isna()]
        if len(null_rows) > 0 and 'SEASON_YEAR' in df_stats.columns:
            # Find most common SEASON_YEAR for these null rows
            common_season = null_rows['SEASON_YEAR'].mode()
            most_common_season.append(common_season.iloc[0] if len(common_season) > 0 else None)
        else:
            most_common_season.append(None)
    else:
        most_common_season.append(None)

na_counts_df = pd.DataFrame({
    'Column': na_counts.index,
    'NA_Count': na_counts.values,
    'NA_Percentage': (na_counts.values / len(df_to_train) * 100).round(2),
    'Most_Common_Season_Year': most_common_season
}).sort_values('NA_Count', ascending=False)

# Show only columns with NAs
na_counts_df[na_counts_df['NA_Count'] > 0]

,Column,NA_Count,NA_Percentage,Most_Common_Season_Year


In [24]:
df_to_train['LINE_ERROR'] = df_to_train['TOTAL_POINTS'] - df_to_train['TOTAL_OVER_UNDER_LINE']

In [25]:
# df_to_train = df_to_train[df_to_train['LINE_ERROR'] != 0]

## Train / Test

In [26]:
X = df_to_train.drop(['TOTAL_POINTS', 'LINE_ERROR', 'SEASON_YEAR'], axis=1, errors='ignore')
y = df_to_train['LINE_ERROR']

In [27]:
# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=16)

In [29]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
# Check number of coulmns
print(f"Number of columns in training set: {X_train.shape[1]}")
print(f"Number of columns in test set: {X_test.shape[1]}")

Training set size: 6119
Test set size: 1080
Number of columns in training set: 909
Number of columns in test set: 909


## Cross-validation

In [30]:
from sklearn.model_selection import KFold, cross_validate, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer, root_mean_squared_error

In [31]:
# Declare KFold
kf = KFold(n_splits=5, shuffle=True, random_state=16)

In [32]:
import numpy as np


def over_under_betting_accuracy(true_error: np.ndarray, pred_error: np.ndarray) -> float:
    true_sign = np.sign(true_error)
    pred_sign = np.sign(pred_error)

    valid = (true_sign != 0) & (pred_sign != 0)
    if not np.any(valid):
        return np.nan

    return float(np.mean(true_sign[valid] == pred_sign[valid]))

class OverUnderScorer:
    """
    Custom sklearn-compatible scorer for over/under betting accuracy.
    """

    def __call__(self, estimator, X, y_true):
        y_pred = estimator.predict(X)

        # betting_line = X["TOTAL_OVER_UNDER_LINE"].to_numpy(dtype=float)

        return over_under_betting_accuracy(
            true_error=np.asarray(y_true, dtype=float),
            pred_error=np.asarray(y_pred, dtype=float),
            # betting_line=betting_line,
        )
over_under_scorer = OverUnderScorer()


In [33]:
# Declare scores to be used
scoring = {
    'MSE': make_scorer(mean_squared_error),
    'RMSE': make_scorer(root_mean_squared_error),
    'MAE': make_scorer(mean_absolute_error),
    'OU_Betting_Accuracy': over_under_scorer,
}

In [34]:
def print_metrics(cv_results):
    for sc in scoring.keys():
        if sc == 'OU_Betting_Accuracy':
            print(f'Train {sc}:', f"{cv_results[f'train_{sc}'].mean():.2%}")
            print(f'Validation {sc}:', f"{cv_results[f'test_{sc}'].mean():.2%}")
        else:
            print(f'Train {sc}:', cv_results[f'train_{sc}'].mean().round(5))
            print(f'Validation {sc}:', cv_results[f'test_{sc}'].mean().round(5))
        print()

In [35]:
def real_vs_pred(model, X_train, y_train):
    preds = cross_val_predict(model, X_train, y_train, cv=kf, n_jobs=-1)
    x_line = np.arange(y_train.min(), y_train.max())
    plt.scatter(y_train, preds)
    plt.plot(x_line, x_line, color='orange')
    plt.xlabel('Real target')
    plt.ylabel('Predicted target')
    plt.show()

## Baseline

In [36]:
from sklearn.dummy import DummyRegressor

season_bl = DummyRegressor(strategy='mean')
cv_results = cross_validate(season_bl, X_train, y_train, cv=kf,
                            scoring=scoring, return_train_score=True)
season_bl.fit(X_train, y_train)
print_metrics(cv_results)

Train MSE: 297.39871
Validation MSE: 297.43082

Train RMSE: 17.24505
Validation RMSE: 17.24294

Train MAE: 13.72045
Validation MAE: 13.72158

Train OU_Betting_Accuracy: 51.85%
Validation OU_Betting_Accuracy: 51.85%



In [37]:
# Baseline 3: Predict the betting line (TOTAL_OVER_UNDER_LINE)
y_pred_baseline_3 = X_train['TOTAL_OVER_UNDER_LINE']

# Evaluate
mse = mean_squared_error(y_train, y_pred_baseline_3)
mae = mean_absolute_error(y_train, y_pred_baseline_3)
rmse = root_mean_squared_error(y_train, y_pred_baseline_3)
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

MSE: 51734.46
RMSE: 227.45
MAE: 226.60


## Logistic Regression

In [38]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
cv_results = cross_validate(lr, X_train.fillna(0), y_train, cv=kf,
                            scoring=scoring, return_train_score=True)

print_metrics(cv_results)

Train MSE: 230.03442
Validation MSE: 344.3919

Train RMSE: 15.16665
Validation RMSE: 18.55382

Train MAE: 11.99775
Validation MAE: 14.72545

Train OU_Betting_Accuracy: 65.53%
Validation OU_Betting_Accuracy: 53.21%



In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

# Example XGBoost regressor:
xgb_reg = XGBRegressor(
    max_depth=4,
    learning_rate=0.02,
    n_estimators=350,
    subsample=0.5,       # Equivalent to max_samples in GBRegressor
    colsample_bytree=0.6, # Equivalent to max_features in GBRegressor
    n_jobs=-2,
    random_state=16)

cv_results = cross_validate(
    xgb_reg, 
    X_train, y_train, 
    cv=kf, 
    scoring=scoring,      # Use your custom scoring or e.g. 'neg_mean_absolute_error'
    return_train_score=True,
    n_jobs=-2
)
# Train final model on full train set
xgb_reg.fit(X_train, y_train)

# Print metrics
print_metrics(cv_results)


Train MSE: 194.71209
Validation MSE: 290.10098

Train RMSE: 13.95354
Validation RMSE: 17.02881

Train MAE: 11.09061
Validation MAE: 13.50428

Train OU_Betting_Accuracy: 79.93%
Validation OU_Betting_Accuracy: 53.38%



In [41]:
feature_importances = xgb_reg.feature_importances_
important_features = np.argsort(feature_importances)[::-1]  
feature_importances = pd.DataFrame({
    'Feature': X_train.columns[important_features],
    'Importance': feature_importances[important_features]
}).sort_values(by="Importance", ascending=False)
feature_importances



,Feature,Importance
0,odds_book_total_line_move_from_opener_caesars,0.002621
1,odds_book_total_line_move_from_opener_fanduel,0.002593
2,odds_book_total_line_move_from_opener_draftkings,0.002423
3,DIFF_FROM_LINE_consensus_opener_LAST_ALL_2_MAT...,0.002400
4,DIFF_FROM_LINE_draftkings_TREND_SLOPE_LAST_5_G...,0.002341
...,...,...
856,TOP5_INJURED_PLAYER_PTS_BEFORE_TEAM_HOME,0.000000
905,odds_spread_home_is_fav_draftkings,0.000000
906,IS_AWAY_WEST_COMFERENCE,0.000000
907,BACK_TO_BACK,0.000000


In [42]:
import numpy as np
import pandas as pd


def error_sign_accuracy(y_true_error, y_pred_error) -> float:
    """
    Same sign of (predicted error) and (true error).

    Push handling:
    - If either sign is 0, exclude that sample from scoring.
      (If you prefer to count (0,0) as correct, I can change it.)
    """
    y_true_error = np.asarray(y_true_error, dtype=float)
    y_pred_error = np.asarray(y_pred_error, dtype=float)

    true_sign = np.sign(y_true_error)
    pred_sign = np.sign(y_pred_error)

    valid = (true_sign != 0) & (pred_sign != 0)
    if not np.any(valid):
        return np.nan

    return float(np.mean(true_sign[valid] == pred_sign[valid]))


def evaluate_error_thresholds(
    model,
    X_test: pd.DataFrame,
    y_test_error: pd.Series,
    thresholds=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10),
):
    """
    Evaluate directional accuracy at different confidence thresholds.

    Threshold rule:
    - Include a game if abs(predicted_error) > t
    """
    # Predict error directly
    y_pred_error = np.asarray(model.predict(X_test), dtype=float)

    margin = np.abs(y_pred_error)

    rows = []
    n_total = len(y_test_error)

    y_true_error_np = y_test_error.to_numpy(dtype=float)

    for t in thresholds:
        mask = margin > t
        n = int(mask.sum())

        acc = (
            np.nan
            if n == 0
            else error_sign_accuracy(y_true_error_np[mask], y_pred_error[mask])
        )

        rows.append(
            {
                "threshold_abs_pred_error_gt": t,
                "n_games": n,
                "pct_of_test": (n / n_total) if n_total else np.nan,
                "directional_accuracy": acc,
            }
        )

    return pd.DataFrame(rows), y_pred_error
results_df, y_pred_test_error = evaluate_error_thresholds(
    model=xgb_reg,
    X_test=X_test,
    y_test_error=y_test,      # y_test must be the REAL ERROR series
    thresholds=range(0, 11),
)

display(
    results_df.style.format(
        {"pct_of_test": "{:.1%}", "directional_accuracy": "{:.2%}"}
    )
)



,threshold_abs_pred_error_gt,n_games,pct_of_test,directional_accuracy
0,0,1080,100.0%,52.01%
1,1,760,70.4%,53.46%
2,2,488,45.2%,58.26%
3,3,299,27.7%,59.12%
4,4,172,15.9%,59.88%
5,5,98,9.1%,69.39%
6,6,56,5.2%,75.00%
7,7,39,3.6%,79.49%
8,8,24,2.2%,91.67%
9,9,19,1.8%,94.74%


# Optuna

In [ ]:
import numpy as np
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

# LINE_COL = "TOTAL_OVER_UNDER_LINE"

# Keep your KFold
kf = KFold(n_splits=5, shuffle=True, random_state=16)

def over_under_betting_accuracy(true_error: np.ndarray, pred_error: np.ndarray) -> float:
    true_sign = np.sign(true_error)
    pred_sign = np.sign(pred_error)

    valid = (true_sign != 0) & (pred_sign != 0)
    if not np.any(valid):
        return np.nan

    return float(np.mean(true_sign[valid] == pred_sign[valid]))


def ou_accuracy_with_threshold_errors(
    true_error: np.ndarray,
    pred_error: np.ndarray,
    threshold_abs: float = 0.0,
    min_coverage: float = 0.25,
):
    # "Edge" in points relative to the line
    margin = np.abs(pred_error)

    # Bet only when predicted edge exceeds threshold
    mask = margin > threshold_abs
    coverage = float(np.mean(mask))

    if coverage < min_coverage or mask.sum() == 0:
        return 0.0, coverage

    score = over_under_betting_accuracy(true_error[mask], pred_error[mask])

    # If everything got filtered as pushes (rare, but possible), penalize
    if np.isnan(score):
        return 0.0, coverage

    return score, coverage

def _predict_best(model, X):
    # Use best iteration if early stopping happened
    if getattr(model, "best_iteration", None) is not None:
        # Newer XGBoost
        try:
            return model.predict(X, iteration_range=(0, model.best_iteration + 1))
        except TypeError:
            # Older compatibility path
            ntree_limit = getattr(model, "best_ntree_limit", None)
            if ntree_limit is not None:
                return model.predict(X, ntree_limit=ntree_limit)
    return model.predict(X)

def objective(trial, X, y):
    threshold_abs = trial.suggest_float("threshold_abs", 0.0, 10.0, step=0.5)
    min_coverage = 0.25

    params = {
        "booster": "gbtree",
        "tree_method": "hist",
        "max_depth": trial.suggest_int("max_depth", 2, 7),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 30.0, log=True),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 0.0075, 0.2, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 50.0, log=True),

        "n_estimators": trial.suggest_int("n_estimators", 250, 450, step=25),
        "early_stopping_rounds": 200,   # moved here
        "eval_metric": "rmse",          # optional, but explicit is good
        "random_state": 16,
        "n_jobs": -1,
    }

    fold_scores = []
    fold_coverages = []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
        X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
        y_tr = y.iloc[tr_idx].to_numpy()
        y_va = y.iloc[va_idx].to_numpy()

        model = XGBRegressor(**params)

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            verbose=False,
        )

        y_pred_error = _predict_best(model, X_va)

        score, coverage = ou_accuracy_with_threshold_errors(
            true_error=y_va,
            pred_error=y_pred_error,
            threshold_abs=threshold_abs,
            min_coverage=min_coverage,
        )

        fold_scores.append(score)
        fold_coverages.append(coverage)

        trial.report(float(np.mean(fold_scores)), step=fold)
        if trial.should_prune():
            raise optuna.TrialPruned()

    trial.set_user_attr("mean_coverage", float(np.mean(fold_coverages)))
    return float(np.mean(fold_scores))

# ----------------------------
# Run the search (2 to 3 hours)
# ----------------------------
# Make sure X_train includes TOTAL_OVER_UNDER_LINE, since you use it in the metric.
# X_train, y_train are your current train split.
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(seed=16),
    pruner=MedianPruner(n_warmup_steps=2),
)

# Time budget: 3 hours. You can adjust to 2*3600 if you want.
study.optimize(lambda t: objective(t, X_train, y_train), timeout= 2*3600, n_jobs=1)

print("Best value (CV OU success):", study.best_value)
print("Best params:")
for k, v in study.best_params.items():
    print(k, v)

# ----------------------------
# Train final model on full train set
# ----------------------------
best_params = study.best_params.copy()
best_threshold = best_params.pop("threshold_abs")  # strategy threshold

final_params = {
    "booster": "gbtree",
    "tree_method": "hist",
    "random_state": 16,
    "n_jobs": -1,
    **best_params,
}

final_model = XGBRegressor(**final_params)

[I 2026-02-17 23:40:37,561] A new study created in memory with name: no-name-42c5bb1a-6aff-4724-ad94-08a7f6b35136
[I 2026-02-17 23:43:20,338] Trial 0 finished with value: 0.22127514231499051 and parameters: {'threshold_abs': 2.0, 'max_depth': 5, 'min_child_weight': 0.8219695696031055, 'gamma': 0.22800975066403162, 'subsample': 0.5525101847434408, 'colsample_bytree': 0.30077284752215994, 'learning_rate': 0.071971944311379, 'reg_alpha': 2.975656697106555e-07, 'reg_lambda': 3.478796625225186e-06, 'n_estimators': 475}. Best is trial 0 with value: 0.22127514231499051.
[I 2026-02-17 23:44:08,030] Trial 1 finished with value: 0.0 and parameters: {'threshold_abs': 5.5, 'max_depth': 2, 'min_child_weight': 3.2561820715318097, 'gamma': 0.7922608676158321, 'subsample': 0.4751969147208164, 'colsample_bytree': 0.36413853049197775, 'learning_rate': 0.07385949950651723, 'reg_alpha': 0.0001507914760359121, 'reg_lambda': 4.526442350789216e-05, 'n_estimators': 350}. Best is trial 0 with value: 0.22127514

Best value (CV OU success): 0.6035747460347686
Best params:
threshold_abs 2.0
max_depth 4
min_child_weight 0.9565760733889516
gamma 2.729861884347252
subsample 0.7244889696318297
colsample_bytree 0.13397225868856136
learning_rate 0.011788203039017325
reg_alpha 7.944935718594158e-05
reg_lambda 1.5294282959442285
n_estimators 450


## Check in Test set

In [50]:
final_model.fit(X_train, y_train, verbose=False)
results_df_final, y_pred_test = evaluate_error_thresholds(
    model=final_model,
    X_test=X_test,
    y_test_error=y_test,
    thresholds=range(0, 11)  # 0..10
)

# Pretty display
display(results_df_final.style.format({
    "pct_of_test": "{:.1%}",
    "error_betting_accuracy": "{:.2%}",
}))


,threshold_abs_pred_error_gt,n_games,pct_of_test,directional_accuracy
0,0,1080,100.0%,0.550982
1,1,671,62.1%,0.557895
2,2,357,33.1%,0.569014
3,3,169,15.6%,0.615385
4,4,83,7.7%,0.674699
5,5,40,3.7%,0.825000
6,6,26,2.4%,0.884615
7,7,19,1.8%,0.894737
8,8,17,1.6%,0.941176
9,9,16,1.5%,1.000000


In [51]:
feature_importances = final_model.feature_importances_
important_features = np.argsort(feature_importances)[::-1]  
feature_importances = pd.DataFrame({
    'Feature': X_train.columns[important_features],
    'Importance': feature_importances[important_features]
}).sort_values(by="Importance", ascending=False)
feature_importances

,Feature,Importance
0,total_caesars_line_over,0.004642
1,odds_book_total_line_move_from_opener_draftkings,0.002947
2,total_draftkings_line_over,0.002918
3,odds_book_total_line_move_from_opener_fanduel,0.002857
4,odds_book_total_line_move_from_opener_caesars,0.002717
...,...,...
891,total_fanduel_price_under,0.000000
905,TOP4_INJURED_PLAYER_DEF_RATING_BEFORE_TEAM_HOME,0.000000
906,TOP6_INJURED_PLAYER_MIN_BEFORE_TEAM_HOME,0.000000
907,TOP6_INJURED_PLAYER_TS_PCT_BEFORE_TEAM_HOME,0.000000


In [47]:

import json
from pathlib import Path

import joblib
import numpy as np
from xgboost import XGBRegressor


def train_final_model(
    X_all,
    y_all,
    study,
    *,
    use_early_stopping: bool = True,
    val_frac: float = 0.1,
    random_state: int = 16,
):
    """
    Train a final XGBRegressor using Optuna best params.
    Also returns the betting threshold (threshold_abs) stored in the study params.
    """
    best_params = study.best_params.copy()
    best_threshold = float(best_params.pop("threshold_abs"))

    final_params = {
        "booster": "gbtree",
        "tree_method": "hist",
        "random_state": random_state,
        "n_jobs": -1,
        "eval_metric": "rmse",
        **best_params,
    }

    model = XGBRegressor(**final_params)

    if not use_early_stopping:
        # Train on all data without early stopping
        model.fit(X_all, y_all, verbose=False)
        return model, best_threshold

    # Keep early stopping by holding out a small validation split
    n = len(X_all)
    rng = np.random.default_rng(random_state)
    idx = np.arange(n)
    rng.shuffle(idx)

    n_val = max(1, int(round(val_frac * n)))
    val_idx = idx[:n_val]
    tr_idx = idx[n_val:]

    X_tr, y_tr = X_all.iloc[tr_idx], y_all.iloc[tr_idx]
    X_va, y_va = X_all.iloc[val_idx], y_all.iloc[val_idx]

    model.set_params(early_stopping_rounds=200)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False,
    )
    return model, best_threshold


def save_model_bundle(
    model: XGBRegressor,
    threshold_abs: float,
    feature_names: list[str],
    out_dir: str | Path,
    name: str = "xgb_ou_model",
):
    """
    Saves:
      - model.joblib (sklearn wrapper)
      - metadata.json (threshold + feature order)
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    model_path = out_dir / f"{name}.joblib"
    meta_path = out_dir / f"{name}.meta.json"

    joblib.dump(model, model_path)

    metadata = {
        "threshold_abs": float(threshold_abs),
        "feature_names": list(feature_names),
        "xgboost_version_note": "Saved via joblib XGBRegressor wrapper",
    }
    meta_path.write_text(json.dumps(metadata, indent=2))

    return model_path, meta_path


def load_model_bundle(model_path: str | Path, meta_path: str | Path):
    model = joblib.load(model_path)
    metadata = json.loads(Path(meta_path).read_text())
    return model, metadata


# -------------------------
# Example usage
# -------------------------

# 1) Build ALL data (replace with your variables)
X_all = X  # your full feature DF, must include TOTAL_OVER_UNDER_LINE column for later scoring/decision rule
y_all = y  # your full target Series

# 2) Train final model
final_model, best_threshold = train_final_model(
    X_all=X_all,
    y_all=y_all,
    study=study,
    use_early_stopping=True,   # or False if you prefer no holdout
    val_frac=0.1,
    random_state=16,
)

# 3) Save model + metadata (threshold + feature ordering)
model_path, meta_path = save_model_bundle(
    model=final_model,
    threshold_abs=best_threshold,
    feature_names=list(X_all.columns),
    out_dir="/home/adrian_alvarez/Projects/NBA_over_under_predictor/models/dif_to_line/",
    name="drop_na_cols_5_all_data_xgb_line_error_14_02_26",
)

print("Saved:", model_path)
print("Saved:", meta_path)

# 4) Load later
loaded_model, meta = load_model_bundle(model_path, meta_path)
print("Loaded threshold_abs:", meta["threshold_abs"])


Saved: /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/dif_to_line/drop_na_cols_5_all_data_xgb_line_error_14_02_26.joblib
Saved: /home/adrian_alvarez/Projects/NBA_over_under_predictor/models/dif_to_line/drop_na_cols_5_all_data_xgb_line_error_14_02_26.meta.json
Loaded threshold_abs: 1.5
